In [ ]:
# Session 26 -  Project 3(Classification Models) by Sreedhara Jagatagar  Sreenivasa

#This data was extracted from the census bureau database found at   
#http://www.census.gov/ftp/pub/DES/www/welcome.html 
#Donor: Ronny Kohavi and Barry Becker,  
#Data Mining and Visualization  
#Silicon Graphics.  
#e-mail: ronnyk@sgi.com for questions. 

#Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).  
#48842 instances, mix of continuous and discrete (train=32561, test=16281)  
#45222 if instances with unknown values are removed (train=30162, test=15060) 

#Duplicate or conflicting instances : 6 
#Class probabilities for adult.all file  
#Probability for the label '>50K' : 23.93% / 24.78% (without unknowns)  
#Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)  

#Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted 
#using the following conditions:  
#((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0)) Prediction task is to determine whether a person makes over 
#50K a year. Conversion of original data as follows:  

#1. Discretized a gross income into two ranges with threshold 50,000.  
#2. Convert U.S. to US to avoid periods.  
#3. Convert Unknown to "?"  
#4. Run MLC++ GenCVFiles to generate data,test.  

#Description of fnlwgt (final weight)The weights on the CPS files are controlled to independent estimates of the 
#civilian noninstitutional population of the US. These are prepared monthly for us by Population Division here at the 
#Census Bureau. We use 3 sets of  controls.

#These are:  
#1. A single cell estimate of the population 16+ for each state.  
#2. Controls for Hispanic Origin by age and sex.  
#3. Controls by Race, age and sex. 

#We use all three sets of controls in our weighting program and "rake" through them 6   times so that by the end 
#we come back to all the controls we used. The term estimate refers to population totals derived from CPS by 
#creating "weighted tallies" of any specified socio-economic characteristics of the population. 
#People with similar demographic characteristics should have similar weights. There is one important caveat to 
#remember about this statement. That is that since the CPS sample is actually a collection of 51 state samples, 
#each with its own probability of selection, the statement  only applies within state.

#Database Link
#https://archive.ics.uci.edu/ml/machine-learning-databases/adult/ 

#Problem 1: 
#Prediction task is to determine whether a person makes over 50K a year. 

#Problem 2: 
#Which factors are important 

#Problem 3: 
#Which algorithms are best for this dataset 

# Import Libraries

In [1]:
#Import required libraries and data: 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

#Import XgBOOST Libraray
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

#%matplotlib inline
import seaborn as sns
sns.set(font_scale = 1.5)

from sklearn.metrics import accuracy_score

# Load Data

In [2]:
#Load Data
#Database Link
#https://archive.ics.uci.edu/ml/machine-learning-databases/adult/ 
#Download both train and test data set into local CENSUS directory.

#Create train and Test Data frame
train_set = pd.read_csv('census/adult.data',header = None) 
test_set = pd.read_csv('census/adult.test', skiprows = 1, header = None) 

#Identify and assign Features Names including both Independant and Target variables.
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 
              'wage_class'] 

#Assign Class Names
train_set.columns = col_labels 
test_set.columns = col_labels 

In [3]:
#Show train data set
train_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
#Shown the test data set
test_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


# Anayze Data

In [5]:
#Show data set information
print(train_set.info())
print(' ')
print(test_set.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
wage_class        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
age               16281 non-null int64
workclass         16281 non-null object
fnlwgt            16281 non-null int64
education    

In [6]:
#Describe the data
print(train_set.describe())
print(' ')
print(test_set.describe())

                age        fnlwgt  education_num  capital_gain  capital_loss  \
count  32561.000000  3.256100e+04   32561.000000  32561.000000  32561.000000   
mean      38.581647  1.897784e+05      10.080679   1077.648844     87.303830   
std       13.640433  1.055500e+05       2.572720   7385.292085    402.960219   
min       17.000000  1.228500e+04       1.000000      0.000000      0.000000   
25%       28.000000  1.178270e+05       9.000000      0.000000      0.000000   
50%       37.000000  1.783560e+05      10.000000      0.000000      0.000000   
75%       48.000000  2.370510e+05      12.000000      0.000000      0.000000   
max       90.000000  1.484705e+06      16.000000  99999.000000   4356.000000   

       hours_per_week  
count    32561.000000  
mean        40.437456  
std         12.347429  
min          1.000000  
25%         40.000000  
50%         40.000000  
75%         45.000000  
max         99.000000  
 
                age        fnlwgt  education_num  capital_gai

In [7]:
#Education wise Wage Class
train_set[['education', 'wage_class']].groupby(['education']).count()

,wage_class
education,
10th,933
11th,1175
12th,433
1st-4th,168
5th-6th,333
7th-8th,646
9th,514
Assoc-acdm,1067
Assoc-voc,1382


In [8]:
#Workclass wise Wage Class
train_set[['workclass', 'wage_class']].groupby(['workclass']).count()

,wage_class
workclass,
?,1836
Federal-gov,960
Local-gov,2093
Never-worked,7
Private,22696
Self-emp-inc,1116
Self-emp-not-inc,2541
State-gov,1298
Without-pay,14


In [9]:
#marital_status wise Wage Class
train_set[['marital_status', 'wage_class']].groupby(['marital_status']).count()

,wage_class
marital_status,
Divorced,4443
Married-AF-spouse,23
Married-civ-spouse,14976
Married-spouse-absent,418
Never-married,10683
Separated,1025
Widowed,993


In [10]:
#occupation wise Wage Class
train_set[['occupation', 'wage_class']].groupby(['occupation']).count()

,wage_class
occupation,
?,1843
Adm-clerical,3770
Armed-Forces,9
Craft-repair,4099
Exec-managerial,4066
Farming-fishing,994
Handlers-cleaners,1370
Machine-op-inspct,2002
Other-service,3295


In [11]:
#relationship wise Wage Class
train_set[['relationship', 'wage_class']].groupby(['relationship']).count()

,wage_class
relationship,
Husband,13193
Not-in-family,8305
Other-relative,981
Own-child,5068
Unmarried,3446
Wife,1568


In [12]:
#race wise Wage Class
train_set[['race', 'wage_class']].groupby(['race']).count()

,wage_class
race,
Amer-Indian-Eskimo,311
Asian-Pac-Islander,1039
Black,3124
Other,271
White,27816


In [29]:
#Sex wise Wage Class
train_set[['sex', 'wage_class']].groupby(['sex']).count()

,wage_class
sex,
Female,10771
Male,21790


In [13]:
#native_country wise Wage Class
train_set[['native_country', 'wage_class']].groupby(['native_country']).count()

,wage_class
native_country,
?,583
Cambodia,19
Canada,121
China,75
Columbia,59
Cuba,95
Dominican-Republic,70
Ecuador,28
El-Salvador,106


# Clean Data

In [15]:
#Clean Train Data -wage_class
print('Before Clean',train_set.wage_class.unique())
train_set['wage_class'] =train_set.wage_class.replace({' <=50K': '<=50K', ' >50K':'>50K'})
print('After Clean',train_set.wage_class.unique())

Before Clean [' <=50K' ' >50K']
After Clean ['<=50K' '>50K']


In [18]:
#Clean Test Data - wage_class
print('Before Clean',test_set.wage_class.unique())
test_set['wage_class'] = test_set.wage_class.replace({' <=50K.': '<=50K', ' >50K.':'>50K'})
print('After Clean ',test_set.wage_class.unique())

Before Clean [' <=50K.' ' >50K.']
After Clean  ['<=50K' '>50K']


In [20]:
#Remove ? Mark Data from Train Set
print('Before Droping the rows which are having ? :',train_set.shape)
train_nomissing = train_set.replace(' ?', np.nan).dropna()
#Remove ? Mark Data from Train Set
print('After Droping the rows which are having ? :',train_nomissing.shape)

Before Droping the rows which are having ? : (32561, 15)
After Droping the rows which are having ? : (30162, 15)


In [21]:
#Remove ? Mark Data from test Set
print('Before Droping the rows which are having ? :',test_set.shape)
test_nomissing = test_set.replace(' ?', np.nan).dropna()
#Remove ? Mark Data from Train Set
print('After Droping the rows which are having ? :',test_nomissing.shape)

Before Droping the rows which are having ? : (16281, 15)
After Droping the rows which are having ? : (15060, 15)


# Convert to Object variable to Numerical Variable for both Test and Train

In [23]:
#Convert object variable to numarical variables for train data set
print('Before Converting ',train_nomissing.info())
for feature in train_nomissing.columns: # Loop through all columns in the dataframe
    if train_nomissing[feature].dtype == 'object': # Only apply for columns with categorical strings
        train_nomissing[feature] = pd.Categorical(train_nomissing[feature]).codes # Replace strings with an integer
print(' ')
print('after Converting ',train_nomissing.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
age               30162 non-null int64
workclass         30162 non-null int8
fnlwgt            30162 non-null int64
education         30162 non-null int8
education_num     30162 non-null int64
marital_status    30162 non-null int8
occupation        30162 non-null int8
relationship      30162 non-null int8
race              30162 non-null int8
sex               30162 non-null int8
capital_gain      30162 non-null int64
capital_loss      30162 non-null int64
hours_per_week    30162 non-null int64
native_country    30162 non-null int8
wage_class        30162 non-null int8
dtypes: int64(6), int8(9)
memory usage: 1.9 MB
Before Converting  None
 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
age               30162 non-null int64
workclass         30162 non-null int8
fnlwgt            30162 non-null int64
education        

In [24]:
#Convert object variable to numarical variables for test data set
#Convert object variable to numarical variables for train data set
print('Before Converting ',test_nomissing.info())
for feature in train_nomissing.columns: # Loop through all columns in the dataframe
    if test_nomissing[feature].dtype == 'object': # Only apply for columns with categorical strings
        test_nomissing[feature] = pd.Categorical(test_nomissing[feature]).codes # Replace strings with an integer
print(' ')
print('after Converting ',test_nomissing.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15060 entries, 0 to 16280
Data columns (total 15 columns):
age               15060 non-null int64
workclass         15060 non-null object
fnlwgt            15060 non-null int64
education         15060 non-null object
education_num     15060 non-null int64
marital_status    15060 non-null object
occupation        15060 non-null object
relationship      15060 non-null object
race              15060 non-null object
sex               15060 non-null object
capital_gain      15060 non-null int64
capital_loss      15060 non-null int64
hours_per_week    15060 non-null int64
native_country    15060 non-null object
wage_class        15060 non-null object
dtypes: int64(6), object(9)
memory usage: 1.8+ MB
Before Converting  None
 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15060 entries, 0 to 16280
Data columns (total 15 columns):
age               15060 non-null int64
workclass         15060 non-null int8
fnlwgt            15060 non-null in

In [25]:
#Assign final data set
final_train =train_nomissing
final_test = test_nomissing

In [26]:
#Describe the Final train data set after cleaning and coverting 
final_train.describe()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
count,30162.000000,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,2.199324,1.897938e+05,10.333764,10.121312,2.580134,5.959850,1.418341,3.678602,0.675685,1092.007858,88.372489,40.931238,36.382567,0.248922
std,13.134665,0.953925,1.056530e+05,3.812292,2.549995,1.498016,4.029566,1.601338,0.834709,0.468126,7406.346497,404.298370,11.979984,6.105372,0.432396
min,17.000000,0.000000,1.376900e+04,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,28.000000,2.000000,1.176272e+05,9.000000,9.000000,2.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,38.000000,0.000000
50%,37.000000,2.000000,1.784250e+05,11.000000,10.000000,2.000000,6.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,38.000000,0.000000
75%,47.000000,2.000000,2.376285e+05,12.000000,13.000000,4.000000,9.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,38.000000,0.000000
max,90.000000,6.000000,1.484705e+06,15.000000,16.000000,6.000000,13.000000,5.000000,4.000000,1.000000,99999.000000,4356.000000,99.000000,40.000000,1.000000


In [27]:
#Describe the Final test data set after cleaning and coverting 
final_test.describe()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
count,15060.000000,15060.000000,1.506000e+04,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,2.214874,1.896164e+05,10.272377,10.112749,2.595286,5.989044,1.401394,3.683665,0.673772,1120.301594,89.041899,40.951594,35.467596,0.245684
std,13.380676,0.966422,1.056150e+05,3.826251,2.558727,1.505336,4.020113,1.588954,0.827630,0.468848,7703.181842,406.283245,12.062831,5.880922,0.430506
min,17.000000,0.000000,1.349200e+04,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,28.000000,2.000000,1.166550e+05,9.000000,9.000000,2.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,37.000000,0.000000
50%,37.000000,2.000000,1.779550e+05,11.000000,10.000000,2.000000,6.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,37.000000,0.000000
75%,48.000000,2.000000,2.385888e+05,12.000000,13.000000,4.000000,9.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,37.000000,0.000000
max,90.000000,6.000000,1.490400e+06,15.000000,16.000000,6.000000,13.000000,5.000000,4.000000,1.000000,99999.000000,3770.000000,99.000000,39.000000,1.000000


# Build the Module

# Evaluate the Module

# Tune the Module

# Answer by Sreedhara Jagatagar  Sreenivasa

Problem 1: 
Prediction task is to determine whether a person makes over 50K a year. 

# Answer by Sreedhara Jagatagar  Sreenivasa

Problem 2: 
Which factors are important

# Answer by Sreedhara Jagatagar  Sreenivasa

Problem 3: 
Which algorithms are best for this dataset 